# Создание датасета

Сначала установлю нужные библиотеки

In [ ]:
%pip install selenium webdriver-manager bs4 pandas surya-ocr

Note: you may need to restart the kernel to use updated packages.


Импортирую зависимости

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time

import os
import time
import re
import pandas as pd
import json

import base64
from bs4 import BeautifulSoup

Код далее инициализирует Selenium WebDriver для Chrome с автоматической загрузкой драйвера, открывает каталог задач ЕГЭ по математике. Кроме того задаются увеличенные таймауты. 

In [ ]:
# --- Настройки ---
chrome_options = Options()
chrome_options.add_argument("--start-maximized")
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

# --- Открываем страницу и настраиваем драйвер  ---
url = "https://mathb-ege.sdamgia.ru/prob-catalog"
driver.get(url)
driver.set_page_load_timeout(2000)
driver.command_executor.set_timeout(2000)

C:\Users\Марина\AppData\Local\Temp\ipykernel_18712\381732346.py:11: DeprecationWarning: set_timeout() in RemoteConnection is deprecated, set timeout in client_config instead
  driver.command_executor.set_timeout(2000)


Функции ниже представляют собой скрипт для автоматизированного скачивания заданий и решений с сайта sdamgia.ru. Он загружает страницы с задачами ЕГЭ по математике, включает отображение решений, извлекает все изображения (формулы и графики) и сохраняет их локально, используя разные методы для Tex-формул (скриншоты) и обычных изображений (JavaScript Canvas). 


**domain** - массив используемых префиксов сайта - отдельно для профильной и базовой математики.

**Функции ниже:**

1. `safe_filename(name)`
**Аргументы:** `name` (str) - исходная URL  
**Возвращает:** str - безопасное имя файла  
**Описание:** Заменяет недопустимые символы в именах файлов на подчеркивания, удаляет префикс https://

2. `safe_get(driver, url, retries=3)`
**Аргументы:** 
- `driver` - экземпляр Selenium WebDriver
- `url` (str) - URL для загрузки
- `retries` (int) - количество попыток при ошибке (по умолчанию 3)  
**Возвращает:** bool - успех загрузки  
**Описание:** Безопасная загрузка страницы с повторными попытками при ошибках

3. `ensure_element_visible(driver, element)`
**Аргументы:**
- `driver` - экземпляр WebDriver
- `element` - элемент Selenium  
**Возвращает:** bool - успех прокрутки  
**Описание:** Прокручивает страницу так, чтобы элемент был полностью виден в окне браузера

4. `download_via_javascript(driver, img_element, filepath)`
**Аргументы:**
- `driver` - экземпляр WebDriver
- `img_element` - элемент изображения
- `filepath` (str) - путь для сохранения файла  
**Возвращает:** bool - успех скачивания  
**Описание:** Скачивает изображение через JavaScript Canvas API

5. `extract_images_from_blocks(driver)`
**Аргументы:** `driver` - экземпляр WebDriver  
**Возвращает:** list - список элементов изображений  
**Описание:** Извлекает все изображения из блоков задач (условия и решения) на странице

6. `enable_solutions_checkbox(driver)`
**Аргументы:** `driver` - экземпляр WebDriver  
**Возвращает:** bool - успех включения  
**Описание:** Включает чекбокс "Показать решения" на странице sdamgia.ru

7. `check_not_visible(element)`
**Аргументы:** `element` - элемент BeautifulSoup  
**Возвращает:** bool - является ли элемент невидимым  
**Описание:** Проверяет, содержит ли элемент классы или стили, указывающие на скрытость

8. `check_all_images_in_file(path, url)`
**Аргументы:** 
- `path` (str) - путь к HTML файлу
- `url` (str) - исходный URL страницы  
**Возвращает:** bool - все ли изображения скачаны  
**Описание:** Проверяет в сохраненном HTML файле, скачаны ли все изображения

9. `download_extracted_images(driver, save_dir="images")`
**Аргументы:** 
- `driver` - экземпляр WebDriver
- `save_dir` (str) - директория для сохранения (по умолчанию "images")  
**Возвращает:** None (печатает статистику)  
**Описание:** Основная функция скачивания картинок - включает отображение решений, извлекает и скачивает все изображения

10. `download_page(driver, url, save_dir="pages", wait=3, load_imgs=False)`
**Аргументы:**
- `driver` - экземпляр WebDriver
- `url` (str) - URL страницы
- `save_dir` (str) - директория для HTML (по умолчанию "pages")
- `wait` (int) - время ожидания после загрузки (по умолчанию 3 сек)
- `load_imgs` (bool) - скачивать ли изображения (по умолчанию False)  
**Возвращает:** str или None - путь к сохраненному файлу  
**Описание:** Полный пайплайн - загружает страницу, сохраняет HTML, при необходимости скачивает изображения

In [ ]:
domain = [
    "https://mathb-ege.sdamgia.ru",
    "https://math-ege.sdamgia.ru"
]

def safe_filename(name): 
    # заменяем все недопустимые символы на _
    return re.sub(r'[<>:"/\\|?*&=.]+', '_', name.replace("https://", ""))

def safe_get(driver, url, retries=3):
    for attempt in range(1, retries+1):
        try:
            driver.get(url)
            return True
        except KeyboardInterrupt:
            print("\n[STOP] Остановка по Ctrl+C.")
            raise 
        except Exception as e:
            print(f"[WARN] Ошибка загрузки (попытка {attempt}/{retries}): {e}")
            time.sleep(3)
    return False

def ensure_element_visible(driver, element):
    """Прокручивает к элементу и делает его полностью видимым."""
    try:
        # Прокручиваем к элементу с центрированием
        driver.execute_script("""
            var element = arguments[0];
            var elementRect = element.getBoundingClientRect();
            var absoluteElementTop = elementRect.top + window.pageYOffset;
            var absoluteElementBottom = elementRect.bottom + window.pageYOffset;
            var middle = absoluteElementTop + (elementRect.height / 2);
            
            // Прокручиваем к середине элемента
            window.scrollTo({
                top: middle - (window.innerHeight / 2),
                behavior: 'smooth'
            });
            
            // Даем время на прокрутку
            return new Promise(resolve => setTimeout(resolve, 500));
        """, element)
        
        # Ждем прокрутки
        time.sleep(0.5)
        
        # Дополнительная проверка видимости
        is_visible = driver.execute_script("""
            var elem = arguments[0];
            var rect = elem.getBoundingClientRect();
            
            // Проверяем, что элемент полностью в видимой области
            return (
                rect.top >= 0 &&
                rect.left >= 0 &&
                rect.bottom <= (window.innerHeight || document.documentElement.clientHeight) &&
                rect.right <= (window.innerWidth || document.documentElement.clientWidth)
            );
        """, element)
        
        if not is_visible:
            # Если не полностью видно, прокручиваем еще
            driver.execute_script("arguments[0].scrollIntoView({block: 'center', inline: 'center', behavior: 'smooth'});", element)
            time.sleep(1)
        
        return True
    
    except Exception as e:
        print(f"Ошибка прокрутки: {e}")
        return False
    
def download_via_javascript(driver, img_element, filepath):
    """Скачивает изображение через JavaScript."""
    try:
        script = """
        var callback = arguments[arguments.length - 1];
        var img = arguments[0];
        
        // Создаем новое изображение
        var newImg = new Image();
        newImg.crossOrigin = 'anonymous';
        newImg.src = img.src || img.currentSrc || img.getAttribute('data-src');
        
        newImg.onload = function() {
            var canvas = document.createElement('canvas');
            canvas.width = this.naturalWidth;
            canvas.height = this.naturalHeight;
            
            var ctx = canvas.getContext('2d');
            ctx.drawImage(this, 0, 0);
            
            try {
                var dataUrl = canvas.toDataURL('image/png');
                callback({success: true, dataUrl: dataUrl});
            } catch(e) {
                callback({success: false, error: 'Canvas error: ' + e});
            }
        };
        
        newImg.onerror = function() {
            callback({success: false, error: 'Image load error'});
        };
        
        // Таймаут
        setTimeout(() => callback({success: false, error: 'Timeout'}), 5000);
        """
        
        result = driver.execute_async_script(script, img_element)
        
        if result.get('success'):
            data_url = result['dataUrl']
            if data_url.startswith('data:image/png;base64,'):
                img_data = base64.b64decode(data_url.split('base64,')[1])
                with open(filepath, 'wb') as f:
                    f.write(img_data)
                return True
        elif result.get('error') != 'Image load error':
            print(f"    JS error: {result.get('error')}")
            
    except Exception as e:
        print(f"    JS exception: {e}")
    
    return False

def extract_images_from_blocks(driver):
    """Извлекает картинки из блоков prob_maindiv."""
    blocks = driver.find_elements(By.CLASS_NAME, "prob_maindiv")
    images = []
    
    for block in blocks:
        try:
            # Изображения из условия
            pbody = block.find_element(By.CLASS_NAME, "pbody")
            images += pbody.find_elements(By.TAG_NAME, "img")
            
            # Изображения из решения
            solution = block.find_element(By.CLASS_NAME, "solution")
            images += solution.find_elements(By.TAG_NAME, "img")
            
        except Exception as e:
            print(f"Ошибка обработки блока: {e}")
    
    return images

def enable_solutions_checkbox(driver):
    """Включает галочку через Selenium поиск."""
    try:        
        # Ищем чекбокс по id
        checkbox = driver.find_element(By.ID, "cb_sol")
        
        if not checkbox.is_selected():
            # Кликаем на сам чекбокс
            checkbox.click()
            time.sleep(1)  # ждем применения
            return True
        else:
            return True
            
    except Exception as e:
        print(f"✗ Не удалось найти чекбокс по id='cb_sol': {e}")
        return False

def check_not_visible(element):
    if "solution" in element.get("class", ''):
        return False
    return ('wtooltip' in element.get('class', '')) or bool(re.search(r'display\s*:\s*none', element.get("style", ''), re.IGNORECASE))

def check_all_images_in_file(path, url):
    """Проверяет что все картинки из файла загружены"""
    with open(path, encoding="utf-8") as f:
        soup = BeautifulSoup(f.read(), "html")

    images = []

    for block in soup.find_all("div", class_="prob_maindiv"):

        for element in block.find_all(check_not_visible):
            element.decompose()
        # тело условия
        images += block.find("div", class_="pbody").find_all("img")
        # решение
        images += block.find("div", class_="solution").find_all("img")

    for img in images:
        if 'display: none' in img.get("styles", ''):
            continue
        src = img.get("src")
        is_tex = "tex" in img.get("class", '')

        if src.startswith('/'):
            if url.startswith(domain[0]):
                src = domain[0] + src
            elif url.startswith(domain[1]):
                src = domain[1] + src
            else:
                print(f"ERROR: неизвестный домен url {url}")
        filename = f"{safe_filename(src)}.png"
        img_path = os.path.join("images", filename)
        if is_tex:
            img_path = os.path.join(os.path.join("images", "tex"), filename)
        if not os.path.exists(img_path):
            return False
    return True

def download_extracted_images(driver, save_dir="images"):
    """Скачивает извлеченные картинки."""
    enable_solutions_checkbox(driver)
    images_data = extract_images_from_blocks(driver)
    os.makedirs(save_dir, exist_ok=True)
    save_dir_tex = os.path.join(save_dir, 'tex')
    os.makedirs(save_dir_tex, exist_ok=True)
    downloaded = 0
    for i, img in enumerate(images_data):
        try:
            src = img.get_attribute("src") or ""
            if not src:
                print("WARNING: нет src в картинке")
                continue
            if not img.is_displayed():
                continue
            
            # Создаем имя файла
            classes = img.get_attribute("class") or ""
            has_tex_class = "tex" in classes.lower()

            filename = safe_filename(src)
            filename = f"{filename}.png"
            filepath = os.path.join(save_dir, filename)
            if has_tex_class:
                filepath = os.path.join(save_dir_tex, filename)
            
            # Пропускаем если уже существует
            if os.path.exists(filepath) and os.path.getsize(filepath) > 0:
                downloaded += 1
                continue
            
            # Скачиваем            
            if has_tex_class:
                if not ensure_element_visible(driver, img):
                    print(f"WARNING: не видно изображение {filename}")
                img.screenshot(filepath)
            else:
                if not download_via_javascript(driver, img, filepath):
                    if not ensure_element_visible(driver, img):
                        print(f"WARNING: не видно изображение {filename}")
                    img.screenshot(filepath)
            
            if os.path.exists(filepath):
                downloaded += 1
                
        except Exception as e:
            print(f"  ✗ Ошибка: {e}")
    print(f"Скачано {downloaded} из {len(images_data)} изображений")

def download_page(driver, url, save_dir="pages", wait=3, load_imgs=False):
    """Скачивает страницу через уже существующий драйвер, если её ещё нет."""

    os.makedirs(save_dir, exist_ok=True)

    # имя файла из URL
    fname = safe_filename(url) + ".html"
    path = os.path.join(save_dir, fname)

    if os.path.exists(path) and (not load_imgs or check_all_images_in_file(path, url)):
        print(f"Файл уже существует: {path}")
        return path

    print(f"Скачиваю страницу {url} ...")
    ok = safe_get(driver, url)
    if not ok:
        print(f"[FAIL] Не удалось загрузить страницу: {url}")
        return None
    time.sleep(wait)

    html = driver.page_source
    with open(path, "w", encoding="utf-8") as f:
        f.write(html)

    print(f"Сохранено: {path}")

    if load_imgs:
        download_extracted_images(driver)
        
    return path

Скачаваю каталоги заданий по темам для базоваого и профильного уровня математики.

In [6]:
_ = download_page(driver, "https://mathb-ege.sdamgia.ru/prob-catalog")
_ = download_page(driver, "https://math-ege.sdamgia.ru/prob-catalog")

Файл уже существует: pages\mathb-ege_sdamgia_ru_prob-catalog.html
Файл уже существует: pages\math-ege_sdamgia_ru_prob-catalog.html


Ячейка ниже содержит функции для парсинга каталога задач с сайта sdamgia.ru. Код обрабатывает сохраненные HTML-страницы каталога, извлекает структуру категорий и подкатегорий задач, очищает текст от форматирования и формирует таблицу с информацией о темах, количестве задач и ссылками для дальнейшей обработки.

**Функции:**

1. `clean_text(text)`
**Аргументы:** `text` (str) - исходный текст  
**Возвращает:** str - очищенный текст  
**Описание:** Удаляет мягкие переносы и лишние пробелы, оставляя только один пробел между словами.

2. `strip_leading_number(text)`
**Аргументы:** `text` (str) - текст с возможным номером в начале  
**Возвращает:** str - текст без ведущего номера  
**Описание:** Удаляет шаблоны вида "число. " в начале строки.

3. `parse_catalog(html_path)`
**Аргументы:** `html_path` (str) - путь к сохраненному HTML-файлу каталога  
**Возвращает:** pandas.DataFrame - таблица с колонками: category, subcategory, link, count  
**Описание:** Парсит структуру каталога задач, извлекает категории и подкатегории, проверяет соответствие количества задач, возвращает структурированные данные для дальнейшей обработки.

In [ ]:
def clean_text(text):
    if not text:
        return text
    # убираем мягкие переносы и лишние пробелы
    text = text.replace("\xad", "")
    return re.sub(r"\s+", " ", text).strip()

def strip_leading_number(text):
    # убирает "1. " или "12. " в начале
    return re.sub(r'^\s*\d+\.\s*', '', text)

def parse_catalog(html_path):
    """
    Возвращает DataFrame с колонками: category, subcategory, link
    (берёт только главные блоки div.Theme.nobg как категории)
    """
    if not os.path.exists(html_path):
        raise FileNotFoundError(html_path)

    with open(html_path, "r", encoding="utf-8") as f:
        soup = BeautifulSoup(f.read(), "html")

    rows = []

    # только главные категории
    for cat_div in soup.select("div.Theme.nobg"):
        # Надёжно достаём всю надпись внутри span.Theme-title
        title_span = cat_div.select_one("span.Theme-title")
        raw_cat = title_span.get_text(" ", strip=True) if title_span else ""
        raw_cat = clean_text(raw_cat)
        cat_name = strip_leading_number(raw_cat)

        # количество задач у категории
        cat_count_tag = cat_div.select_one("span.Counter.Counter")
        cat_count = int(cat_count_tag.text.strip()) if cat_count_tag else None

        # главная ссылка (на "Перейти" у категории)
        cat_link_tag = cat_div.select_one("a.Theme-link")
        cat_link = cat_link_tag.get("href") if cat_link_tag else None

        # дети (подкатегории) — только те, что внутри Theme-children
        children_div = cat_div.select_one("div.Theme-children")
        sub_counts = []
        if children_div:
            for sub in children_div.select("div.Theme"):
                # подкатегория обычно в a.Theme-titlenot
                sub_name_tag = sub.select_one("a.Theme-titlenot")
                # но на всякий случай fallback на span.Theme-title внутри
                if not sub_name_tag:
                    sub_name_tag = sub.select_one("span.Theme-title")
                sub_name = clean_text(sub_name_tag.get_text(" ", strip=True)) if sub_name_tag else None

                sub_link_tag = sub.select_one("a.Theme-link")
                sub_link = sub_link_tag.get("href") if sub_link_tag else None

                sub_count_tag = sub.select_one("span.Counter.Counter")
                sub_count = int(sub_count_tag.text.strip()) if sub_count_tag else None
                sub_counts.append(sub_count)
                
                rows.append({
                    "category": cat_name,
                    "subcategory": sub_name,
                    "link": sub_link,
                    "count": sub_count
                })

            # проверка: сумма подкатегорий == count категории
            sum_sub = sum(sub_counts)
            if cat_count is not None and sum_sub != cat_count:
                print(f"[WARN] Несовпадение count для категории '{cat_name}': {cat_count} != сумма подкатегорий {sum_sub}")
        else:
            # категория без подкатегорий — записываем саму категорию
            rows.append({
                "category": cat_name,
                "subcategory": None,
                "link": cat_link,
                "count": cat_count
            })

    return pd.DataFrame(rows)


Провожу парсинг каталогов задач для профильного и базового уровня, формирую общий датасет с дополнительной колонкой profile и сохраняю его.

In [66]:
files = [
    ("pages\math-ege_sdamgia_ru_prob-catalog.html", "profile"),
    ("pages\mathb-ege_sdamgia_ru_prob-catalog.html", "basic")
]
df_list = []

for file_path, profile_type in files:
    df = parse_catalog(file_path)
    df['profile'] = profile_type
    df_list.append(df)

# объединяем
all_df = pd.concat(df_list, ignore_index=True)

# сохраняем в CSV
all_df.to_csv("ege_categories.csv", encoding="utf-8")

print("Готово! CSV создан: ege_categories.csv")

Готово! CSV создан: ege_categories.csv


Код ниже реализует пайплайн массового скачивания страниц с задачами из подготовленного каталога. Он использует предварительно собранную таблицу категорий, формирует правильные URL для версий "print" (без лишних элементов интерфейса) и последовательно загружает все страницы, сохраняя HTML и при необходимости изображения.

**Функции и переменные:**

1. Константа `BASE_URLS`
**Тип:** dict  
**Описание:** Словарь с базовыми URL двух версий сайта ЕГЭ по математике: профильный и базовый уровень.

2. Функция `download_all_print_pages(driver, df, save_dir="pages", load_imgs=False)`
**Аргументы:**
- `driver` - экземпляр Selenium WebDriver для управления браузером
- `df` (DataFrame) - таблица с категориями задач для скачивания
- `save_dir` (str) - директория для сохранения HTML страниц (по умолчанию "pages")
- `load_imgs` (bool) - флаг скачивания изображений (по умолчанию False)

**Возвращает:** pandas.DataFrame - обновленная таблица с путями к сохраненным файлам и статусами загрузки

**Описание:** 
Основная функция массового скачивания, которая:
1. Добавляет в таблицу служебные колонки для отслеживания прогресса
2. Для каждой строки таблицы формирует URL в формате "print" (упрощенная версия для печати)
3. Вызывает функцию `download_page` для загрузки и сохранения страницы
4. Обновляет таблицу путями к сохраненным файлам и статусом скачивания изображений
5. Выводит прогресс в консоль (номер текущей страницы из общего количества)

In [ ]:
BASE_URLS = {
    "profile": "https://math-ege.sdamgia.ru",
    "basic":     "https://mathb-ege.sdamgia.ru"
}

def download_all_print_pages(driver, df, save_dir="pages", load_imgs=False):
    if 'file' not in df.columns:
        df['file'] = None
    if 'downloaded_imgs' not in df.columns:
        df['downloaded_imgs'] = False

    for idx, row in df.iterrows():
        print(f"{idx + 1}/{len(df)}")
        base_url = BASE_URLS[row["profile"]]
        
        # ссылка из таблицы
        link = row["link"]
        
        # добавляем print=true корректно
        if "print=true" in link:
            link_print = link
        elif "?" in link:
            link_print = link + "&print=true"
        else:
            link_print = link + "?print=true"

        full_url = base_url + link_print

        save_path = download_page(driver, full_url, save_dir, load_imgs=load_imgs and not df.loc[idx, 'downloaded_imgs'])

        df.loc[idx, 'file'] = save_path
        df.loc[idx, 'downloaded_imgs'] = load_imgs or df.loc[idx, 'downloaded_imgs']

    return df

Скачаваю все страницы и изображения с них и сохраняю итоговый датасет.

In [74]:
all_df = download_all_print_pages(driver, all_df, load_imgs=True)
# сохраняем в CSV
all_df.to_csv("ege_categories.csv", encoding="utf-8")

1/372
Файл уже существует: pages\math-ege_sdamgia_ru_test_category_id_79_filter_all_print_true.html
2/372
Файл уже существует: pages\math-ege_sdamgia_ru_test_category_id_90_filter_all_print_true.html
3/372
Файл уже существует: pages\math-ege_sdamgia_ru_test_category_id_96_filter_all_print_true.html
4/372
Файл уже существует: pages\math-ege_sdamgia_ru_test_category_id_102_filter_all_print_true.html
5/372
Файл уже существует: pages\math-ege_sdamgia_ru_test_category_id_94_filter_all_print_true.html
6/372
Файл уже существует: pages\math-ege_sdamgia_ru_test_category_id_111_filter_all_print_true.html
7/372
Файл уже существует: pages\math-ege_sdamgia_ru_test_category_id_112_filter_all_print_true.html
8/372
Файл уже существует: pages\math-ege_sdamgia_ru_test_category_id_113_filter_all_print_true.html
9/372
Файл уже существует: pages\math-ege_sdamgia_ru_test_category_id_114_filter_all_print_true.html
10/372
Файл уже существует: pages\math-ege_sdamgia_ru_test_category_id_182_filter_all_print_tru

Ниже можно увидеть 10 случайных строк датасете с категориями. 

**Колонки датасета:**
1. **category** — Основная тематическая категория задач 

2. **subcategory** — Подраздел внутри категории с более узкой тематикой 

3. **link** — Относительный URL-путь к странице с задачами данной подкатегории 

4. **count** — Количество задач в данной подкатегории

5. **profile** — Уровень сложности: "basic" (базовый уровень ЕГЭ) или "profile" (профильный уровень ЕГЭ)

In [ ]:
all_df.sample(10)

,category,subcategory,link,count,profile
94,Стереометрическая задача,Угол между скрещивающимися прямыми,/test?category_id=285&filter=all,27,profile
124,Планиметрическая задача,Вписанные окружности и четырехугольники,/test?category_id=279&filter=all,36,profile
253,Планиметрия,Четырёхугольники и их элементы,/test?category_id=288&filter=all,22,basic
78,Уравнения,"Тригонометрические уравнения, разложение на мн...",/test?category_id=291&filter=all,61,profile
282,Текстовые задачи,Задачи на совместную работу,/test?category_id=87&filter=all,27,basic
245,Прикладная стереометрия,Куб,/test?category_id=249&filter=all,3,basic
15,Стереометрия,Призма,/test?category_id=178&filter=all,52,profile
264,Вычисления,Действия с обыкновенными дробями,/test?category_id=55&filter=all,28,basic
214,Размеры и единицы измерения,Единицы измерения массы,/test?category_id=244&filter=all,10,basic
368,Задания Д18. Наибольшее и наименьшее значение ...,Исследование тригонометрических функций,/test?category_id=78&filter=all,29,basic


Для автоматического распознавания математических формул в изображениях и преобразования их в LaTeX-формат я использовала специализированный инструмент Surya OCR (https://github.com/datalab-to/surya), оптимизированный для работы с математическими обозначениями. Вывода здесь нет, так как я запускала его на сервере с GPU.

In [ ]:
!surya_latex_ocr ./images/tex/ 

Код ниже реализует пайплайн обработки сохраненных HTML-страниц с задачами: преобразует математические формулы из изображений в LaTeX-формат, извлекает текстовые условия и решения, собирает все в структурированный датасет. Код работает с двумя версиями текста — исходной русской и преобразованной с LaTeX-формулами.

**Функции:**

1. `load_formulas(json_path)`
**Аргументы:** `json_path` (str) - путь к JSON-файлу с формулами  
**Возвращает:** dict - словарь {имя файла: LaTeX-формула}  
**Описание:** Загружает предварительно извлеченные математические формулы из JSON, преобразует XML-формат формул в LaTeX (заменяет `<math>` теги на `$$`).

2. `normalize_russian_math_text(s)`
**Аргументы:** `s` (str) - исходный текст  
**Возвращает:** str - очищенный текст  
**Описание:** Удаляет специальные Unicode-символы (мягкие переносы, тонкие пробелы, неразрывные пробелы), мешающие обработке.

3. `parse_paragraphs(block)`
**Аргументы:** `block` - элемент BeautifulSoup  
**Возвращает:** str - нормализованный текст с переносами строк  
**Описание:** Извлекает текст из всех параграфов элемента, добавляет переносы строк и нормализует специальные символы.

4. `process_imgs(element, formulas, url)`
**Аргументы:**
- `element` - элемент BeautifulSoup
- `formulas` (dict) - словарь LaTeX-формул
- `url` (str) - исходный URL страницы  
**Возвращает:** tuple - (LaTeX-текст, русский текст, список путей к изображениям)  
**Описание:** Заменяет изображения формул на соответствующие LaTeX или на alt аргумен изображения, обычные изображения — на пути к файлам, возвращает две версии текста.

5. `parse_problem_html(path, url, formulas)`
**Аргументы:**
- `path` (str) - путь к HTML-файлу
- `url` (str) - исходный URL
- `formulas` (dict) - словарь LaTeX-формул  
**Возвращает:** pandas.DataFrame - таблица с задачами  
**Описание:** Парсит HTML-файл с задачами, извлекает условия и решения, заменяет формулы на LaTeX.

6. `parse_dataset(df, save_path="all_problems.csv")`
**Аргументы:**
- `df` (DataFrame) - таблица с метаданными и путями к HTML-файлам
- `save_path` (str) - путь для сохранения итогового датасета (по умолчанию "all_problems.csv")  
**Возвращает:** pandas.DataFrame - объединенный датасет всех задач  
**Описание:** Основная функция обработки — последовательно парсит все HTML-файлы, объединяет результаты, проверяет соответствие количества задач, сохраняет итоговый датасет.

In [ ]:
def load_formulas(json_path):
    """Загружает JSON с формулами и возвращает словарь key → latex_formula."""
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    formulas = {}

    for key, value in data.items():
        if not value or "equation" not in value[0]:
            continue

        eq = value[0]["equation"]

        # Удаляем <math display="block"> и </math>, превращая в $$
        eq = re.sub(r'<math[^>]*>', '$$ ', eq)
        # убираем закрывающий тег
        eq = eq.replace('</math>', ' $$')

        formulas[key] = eq

    return formulas

def normalize_russian_math_text(s):
    # удалить мягкие переносы, тонкие пробелы, нулевые ширины
    chars = [
        "\u00AD",  # soft hyphen
        "\u200B",  # zero-width space
        "\u2060",  # word joiner
        "\xa0",    # non-breaking space
    ]
    for ch in chars:
        s = s.replace(ch, "")
    s = s.replace("\u202f", ' ')
    return s

def parse_paragraphs(block):
    for p in block.find_all("p"):
        # добавляем в конец каждого <p> символ переноса строки
        p.append("\n")
    return normalize_russian_math_text(block.get_text('', strip=False))

def process_imgs(element, formulas, url):
    imgs = []
    element_latex = element.__copy__()  # копия для LaTeX
    
    for img_rus, img_tex in zip(element.find_all("img"), element_latex.find_all("img")):
        is_tex = 'tex' in img_rus.get("class", "")
        src = img_rus.get("src")

        if src.startswith('/'):
            if url.startswith(domain[0]):
                src = domain[0] + src
            elif url.startswith(domain[1]):
                src = domain[1] + src
            else:
                print(f"ERROR: неизвестный домен url {url}")
        filename = safe_filename(src)
        img_path = os.path.join("images", f"{safe_filename(src)}.png")

        if is_tex:
            img_tex.replace_with(formulas[filename])

            alt = img_rus.get("alt", '').strip()
            img_rus.replace_with(alt)
        else:
            img_tex.replace_with(' ')
            img_rus.replace_with(' ')
            imgs.append(img_path)

    return parse_paragraphs(element_latex), parse_paragraphs(element), imgs

def parse_problem_html(path, url, formulas):
    with open(path, encoding="utf-8") as f:
        soup = BeautifulSoup(f.read(), "html")

    rows = []

    for block in soup.find_all("div", class_="prob_maindiv"):
        for element in block.find_all(check_not_visible):
            element.decompose()

        # id задачи (типо maindiv641019)
        body_id = block.get("id")
        num = re.sub(r"\D+", "", body_id)

        # тело условия
        body = block.find("div", class_="pbody")
        body_text, body_text_rus, imgs_b = process_imgs(body, formulas, url)

        # решение
        sol_block = block.find("div", class_="solution")
        sol_text, sol_text_rus, imgs_s = process_imgs(sol_block, formulas, url)

        rows.append({
            "id": num,
            "condition": body_text,
            "images_condition": imgs_b,
            "solution": sol_text,
            "condition_rus": body_text_rus,
            "solution_rus": sol_text_rus,
            "images_solution": imgs_s,
        })

    return pd.DataFrame(rows)

def parse_dataset(df, save_path="all_problems.csv"):
    """
    df: исходный DataFrame с колонками category, subcategory, link, count, profile, file
    save_path: куда сохранить итоговый объединённый датасет
    """
    all_dfs = []
    formulas = load_formulas("results.json")

    for idx, row in df.iterrows():
        html_file = row['file']
        print(f"[{idx+1}/{len(df)}] Парсим {html_file} {row['subcategory']}")
        base_url = BASE_URLS[row["profile"]]
        url = base_url + row["link"] + "?print=true"
        problems_df = parse_problem_html(html_file, formulas=formulas, url=url)

        # проверяем число задач
        expected_count = row.get("count")
        actual_count = len(problems_df)
        if expected_count is not None and actual_count != expected_count:
            print(f"[ERROR] {html_file}: ожидается {expected_count} задач, найдено {actual_count}")
            return None

        # добавляем данные из исходного датафрейма
        for col in ['category', 'subcategory', 'link', 'profile', 'file']:
            if col in row:
                problems_df[col] = row[col]

        all_dfs.append(problems_df)

    # объединяем все
    if all_dfs:
        final_df = pd.concat(all_dfs, ignore_index=True)
    else:
        final_df = pd.DataFrame()

    # сохраняем
    final_df.to_csv(save_path, index=False)
    print(f"[INFO] Итоговый датасет сохранён в {save_path}, всего задач: {len(final_df)}")
    
    return final_df

Создаю датасет задач и решений

In [53]:
df = parse_dataset(all_df)

[1/372] Парсим pages\math-ege_sdamgia_ru_test_category_id_79_filter_all_print_true.html Решение прямоугольного треугольника
[2/372] Парсим pages\math-ege_sdamgia_ru_test_category_id_90_filter_all_print_true.html Решение равнобедренного треугольника
[3/372] Парсим pages\math-ege_sdamgia_ru_test_category_id_96_filter_all_print_true.html Треугольники общего вида
[4/372] Парсим pages\math-ege_sdamgia_ru_test_category_id_102_filter_all_print_true.html Параллелограммы
[5/372] Парсим pages\math-ege_sdamgia_ru_test_category_id_94_filter_all_print_true.html Трапеция
[6/372] Парсим pages\math-ege_sdamgia_ru_test_category_id_111_filter_all_print_true.html Центральные и вписанные углы
[7/372] Парсим pages\math-ege_sdamgia_ru_test_category_id_112_filter_all_print_true.html Касательная, хорда, секущая
[8/372] Парсим pages\math-ege_sdamgia_ru_test_category_id_113_filter_all_print_true.html Вписанные окружности
[9/372] Парсим pages\math-ege_sdamgia_ru_test_category_id_114_filter_all_print_true.html Оп

Ниже можно увидеть 10 случайных колонок датасета с задачами.

**Колонки:**
1. **id** — Уникальный числовой идентификатор задачи, извлеченный из HTML-элемента
2. **condition** — Условие задачи с математическими формулами в LaTeX-формате 
3. **images_condition** — Список путей к обычным изображениям (не формулам) в условии задачи
4. **solution** — Решение задачи с математическими формулами в LaTeX-формате
5. **condition_rus** — Оригинальное условие задачи на русском языке (формулы как текст на русском)
6. **solution_rus** — Оригинальное решение задачи на русском языке
7. **images_solution** — Список путей к обычным изображениям в решении задачи
8. **category** — Основная тематическая категория 
9. **subcategory** — Подраздел внутри категории 
10. **link** — Относительный URL к странице с задачами этой подкатегории
11. **profile** — Уровень сложности: "basic" (базовый ЕГЭ) или "profile" (профильный ЕГЭ)
12. **file** — Локальный путь к сохраненному HTML-файлу с задачами подкатегории

In [76]:
df.sample(10)

,id,condition,images_condition,solution,condition_rus,solution_rus,images_solution,category,subcategory,link,profile,file
10172,906049,Найдите точку максимума функции $$ y = 11^{6x-...,[],\nРешение. Поскольку функция $$ y = 11^{x} $$ ...,Найдите точку максимума функции y=11 в степени...,\nРешение. Поскольку функция y=11 в степени x ...,[],Задания Д18. Наибольшее и наименьшее значение ...,Исследование функций без помощи производной,/test?category_id=175&filter=all,basic,pages\mathb-ege_sdamgia_ru_test_category_id_17...
9997,695706,\nПлощадь основания конуса равна 18. Плоскост...,[images\mathb-ege_sdamgia_ru_get_file_id_13999...,"\nРешение. Сечение плоскостью, параллельной ос...",\nПлощадь основания конуса равна 18. Плоскост...,"\nРешение. Сечение плоскостью, параллельной ос...",[],Задания Д16. Задачи по стереометрии,Конус,/test?category_id=144&filter=all,basic,pages\mathb-ege_sdamgia_ru_test_category_id_14...
9309,918097,Основания трапеции равны 4 и 10. Найдите боль...,[images\mathb-ege_sdamgia_ru_get_file_id_11333...,\nРешение. Больший отрезок средней линии трапе...,Основания трапеции равны 4 и 10. Найдите боль...,\nРешение. Больший отрезок средней линии трапе...,[],Задания Д15. Планиметрия,Трапеция: длины и площади,/test?category_id=124&filter=all,basic,pages\mathb-ege_sdamgia_ru_test_category_id_12...
9099,945529,\nВ правильной треугольной пирамиде SABC с ве...,[images\mathb-ege_sdamgia_ru_get_file_id_11172...,\nРешение. Отрезок OS высота треугольной пирам...,\nВ правильной треугольной пирамиде SABC с ве...,\nРешение. Отрезок OS высота треугольной пирам...,[],Задания Д13. Стереометрия,Пирамида,/test?category_id=177&filter=all,basic,pages\mathb-ege_sdamgia_ru_test_category_id_17...
6435,643845,Две окружности пересекаются в точках A и B. Че...,[],\nРешение. Пусть центры окружностей O1 и O2....,Две окружности пересекаются в точках A и B. Че...,\nРешение. Пусть центры окружностей O1 и O2....,[images\math-ege_sdamgia_ru_get_file_id_30354....,Задания Д14 C4. Многоконфигурационная планимет...,Окружности и системы окружностей,/test?category_id=216&filter=all,profile,pages\math-ege_sdamgia_ru_test_category_id_216...
3437,420536,Две окружности касаются внешним образом в точк...,[],\nРешение. а) Обозначим центры окружностей...,Две окружности касаются внешним образом в точк...,\nРешение. а) Обозначим центры окружностей...,[images\math-ege_sdamgia_ru_get_file_id_148088...,Планиметрическая задача,Окружности и системы окружностей,/test?category_id=277&filter=all,profile,pages\math-ege_sdamgia_ru_test_category_id_277...
5657,636366,В правильной треугольной пирамиде SABC сторона...,[],"\nРешение. а) Докажем, что прямая SC перпенд...",В правильной треугольной пирамиде SABC сторона...,"\nРешение. а) Докажем, что прямая SC перпенд...",[images\math-ege_sdamgia_ru_get_file_id_41452....,Задания Д10 C2. Сложная стереометрия,Круглые тела,/test?category_id=231&filter=all,profile,pages\math-ege_sdamgia_ru_test_category_id_231...
467,919344,Найдите объем правильной шестиугольной призмы...,[images\math-ege_sdamgia_ru_get_file_id_113235...,"\nРешение. Объем прямой призмы равен V = Sh,...",Найдите объем правильной шестиугольной призмы...,"\nРешение. Объем прямой призмы равен V = Sh,...",[],Стереометрия,Призма,/test?category_id=178&filter=all,profile,pages\math-ege_sdamgia_ru_test_category_id_178...
1819,424242,а) Решите уравнение $$ 2\sin 2x - \sin x \cdo...,[],"\nРешение. а) Заметим, что \n\n$$ x = \begin{...",а) Решите уравнение 2 синус 2x минус синус x ...,"\nРешение. а) Заметим, что \n\nx= система выр...",[images\math-ege_sdamgia_ru_get_file_id_46175....,Уравнения,Тригонометрия и иррациональности,/test?category_id=201&filter=all,profile,pages\math-ege_sdamgia_ru_test_category_id_201...
3578,636710,Квадрат ABCD вписан в окружность. Хорда CE пер...,[],\nРешение. а) В треугольниках CKD и CDE угол...,Квадрат ABCD вписан в окружность. Хорда CE пер...,\nРешение. а) В треугольниках CKD и CDE угол...,[images\math-ege_sdamgia_ru_get_file_id_150092...,Планиметр